# Cross-entropy kernel fit

In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path('..') / 'src'))
from kl_decomposition import rectangle_rule
from kl_decomposition.kernel_fit import fit_exp_sum_ce
import numpy as np

In [ ]:
import numpy as np
from numba import njit, prange


@njit(parallel=True, fastmath=True)
def objective_numba(params: np.ndarray,
                    d: np.ndarray,
                    target: np.ndarray,
                    w: np.ndarray) -> np.ndarray:
    """
    Vectorised objective for K parameter sets.

    Parameters
    ----------
    params  : (K, 2*n_terms) log-space parameters
    d       : (N,)           data values (distances); will use d**2
    target  : (N,)           reference curve
    w       : (N,)           weights

    Returns
    -------
    obj_val : (K,)           objective for each parameter row
    """
    K, P = params.shape
    n_terms = P // 2
    N = d.size
    d2 = d * d                      # pre-compute d²
    out = np.empty(K, dtype=params.dtype)

    for k in prange(K):                  # loop over parameter sets in parallel
        # split, exponentiate (back to linear scale)
        a = np.exp(params[k, :n_terms])
        b = np.exp(params[k, n_terms:])

        acc = 0.0                        # accumulate Σ w*(pred-target)²
        for j in range(N):               # loop over data points
            dj2 = d2[j]
            predj = 0.0
            for i in range(n_terms):     # Σ_i a_i * exp(-b_i * d²)
                predj += a[i] * np.exp(-b[i] * dj2)
            diff = predj - target[j]
            acc += w[j] * diff * diff

        out[k] = acc

    return out

In [ ]:
import numpy as np
from typing import Callable, Tuple, Dict, Any, Optional


def cross_entropy(
    user_fn: Callable[[np.ndarray], Tuple[np.ndarray, np.ndarray, Dict[str, Any]]],
    initial_mean: np.ndarray,
    initial_std: np.ndarray,
    pop_size: int = 100,
    elite_frac: float = 0.2,
    alpha_mean: float = 0.7,
    alpha_std: float = 0.7,
    n_iters: int = 100,
    random_state: Optional[int] = None,
    verbose: bool = False,
    tol_std: float = 1e-6,
    tol_g: float = 1e-6
) -> Tuple[np.ndarray, float, Dict[str, Any]]:
    """
    Cross-Entropy Method with independent Gaussian distribution and stopping criteria.

    Args:
        user_fn: function taking samples (shape Nxd) and returning
            (updated_samples, scores, info_dict)
        initial_mean: initial mean vector (d,)
        initial_std: initial std deviation vector (d,)
        pop_size: number of samples per iteration
        elite_frac: fraction of samples selected as elites
        alpha_mean: learning rate for mean update
        alpha_std: learning rate for std update
        n_iters: maximum number of iterations
        random_state: RNG seed for reproducibility
        verbose: print debug info each iteration
        tol_std: stop if max(std) <= tol_std
        tol_g: stop if info_dict['normg'] <= tol_g

    Returns:
        best_sample: best found sample (d,)
        best_score: best objective value
        info: dict with:
            'iter': iteration index at stop
            'best_score_history': list of best_score after each iteration
            'max_std_history': list of max_std after each iteration
    """
    rng = np.random.RandomState(random_state)
    mean = np.array(initial_mean, dtype=float)
    std = np.array(initial_std, dtype=float)
    n_elite = max(1, int(np.ceil(pop_size * elite_frac)))
    best_sample: Optional[np.ndarray] = None
    best_score = np.infb

    # history lists
    best_score_history: list = []
    max_std_history: list = []

    for iteration in range(n_iters):
        samples = rng.randn(pop_size, mean.size) * std + mean
        samples, scores, info_dict = user_fn(samples)
        idx = int(np.argmin(scores))
        # update best if improved
        if scores[idx] < best_score:
            best_score = float(scores[idx])
            best_sample = samples[idx]
        # select elites and update distribution
        elite = samples[np.argsort(scores)[:n_elite]]
        elite_mean = elite.mean(axis=0)
        elite_std = elite.std(axis=0, ddof=0)
        mean = alpha_mean * elite_mean + (1 - alpha_mean) * mean
        std = alpha_std * elite_std + (1 - alpha_std) * std
        # record history
        max_std = float(np.max(std))
        best_score_history.append(best_score)
        max_std_history.append(max_std)
        # debug print
        if verbose:
            print(f"Iter {iteration}: best_score={best_score}, max_std={max_std}, info={info_dict}")
        # stopping criteria
        if max_std <= tol_std:
            break
        if info_dict.get('normg', np.inf) <= tol_g:
            break

    info = {
        'iter': iteration,
        'best_score_history': best_score_history,
        'max_std_history': max_std_history
    }
    return best_sample, best_score, info

In [ ]:
import numpy as np
import jax
import jax.numpy as jnp
from kl_decomposition.kernel_fit import rectangle_rule

# Enable 64-bit (double) precision in JAX
jax.config.update("jax_enable_x64", True)

# setup squared exponential approximation
x, w = rectangle_rule(0.0, 2.0, 500)
target = np.exp(-x)

# Convert to JAX arrays with float64 dtype
x_j = jnp.array(x, dtype=jnp.float64)
w_j = jnp.array(w, dtype=jnp.float64)
target_j = jnp.array(target, dtype=jnp.float64)


def obj_jax(p):
    # p is already float64
    n_param = p.shape[0] // 2
    a = p[:n_param]
    b = p[n_param:]
    # compute prediction and weighted RMS error
    pred = jnp.sum(a[:, None] * jnp.exp(-b[:, None] * x_j[None, :] ** 2), axis=0)
    diff = pred - target_j
    return jnp.sqrt(jnp.sum(w_j * diff ** 2))


# gradients and Hessians in double precision
grad_jax = jax.jit(jax.grad(obj_jax))
hess_jax = jax.jit(jax.hessian(obj_jax))
obj_jax_jit = jax.jit(obj_jax)


def obj(p):
    # ensure NumPy→JAX conversion to float64
    return float(obj_jax_jit(jnp.array(p, dtype=jnp.float64)))


def grad(p):
    return np.array(grad_jax(jnp.array(p, dtype=jnp.float64)))


def hess(p):
    return np.array(hess_jax(jnp.array(p, dtype=jnp.float64)))

In [ ]:
import numpy as np
def golden_section_search(f, a=0.0, b=1.0, tol=1e-6, max_iter=100):
    """
    Golden-section search to find the minimum of a unimodal function f on [a, b].

    Parameters
    ----------
    f : callable
        The objective function to minimize.
    a : float
        Left endpoint of the initial interval.
    b : float
        Right endpoint of the initial interval.
    tol : float
        Tolerance for the interval width (stopping criterion).
    max_iter : int
        Maximum number of iterations.

    Returns
    -------
    x_min : float
        Estimated position of the minimum.
    f_min : float
        Value of f at x_min.
    """
    # Golden ratio constant
    phi = (np.sqrt(5.0) - 1) / 2

    # Initialize interior points
    c = b - phi * (b - a)
    d = a + phi * (b - a)
    f_c = f(c)
    f_d = f(d)

    for _ in range(max_iter):
        if abs(b - a) < tol:
            break

        if f_c < f_d:
            b, d, f_d = d, c, f_c
            c = b - phi * (b - a)
            f_c = f(c)
        else:
            a, c, f_c = c, d, f_d
            d = a + phi * (b - a)
            f_d = f(d)

    # Choose the best of the final points
    if f_c < f_d:
        x_min, f_min = c, f_c
    else:
        x_min, f_min = d, f_d
    # print(f"Iterations: {_ + 1}")
    return x_min, f_min


def newton(f, grad, hess, x0,
                                   tol_grad=1e-6,
                                   tol_step=1e-8,
                                   stall_iter=5,
                                   max_iter=100,
                                   ls_bounds=(-1.0, 1.0),
                                   verbose=False):
    """
    Newton's method with backtracking line search and fallback to gradient descent.

    Parameters
    ----------
    f : callable
        Objective function f(x).
    grad : callable
        Gradient of f: grad(x).
    hess : callable
        Hessian of f: hess(x).
    x0 : ndarray
        Initial guess.
    tol_grad : float, optional
        Tolerance for gradient norm.
    tol_step : float, optional
        Tolerance for step size norm for stall criterion.
    stall_iter : int, optional
        Number of consecutive small steps to trigger stop.
    max_iter : int, optional
        Maximum number of iterations.
    ls_bounds : (float, float), optional
        Initial interval for line search.

    Returns
    -------
    x : ndarray
        Estimated minimizer.
    grad_norm : float
        Norm of gradient at the solution.
    f_history : list of float
        History of f values.
    grad_norm_history : list of float
        History of gradient norms.
    """
    x = x0.astype(float)
    f_history = []
    grad_norm_history = []
    fx = f(x)

    stall_count = 0
    trust = 1e-6 * np.eye(len(x))  # trust region to ensure positive definiteness

    for k in range(max_iter):
        gx = grad(x)
        grad_norm = np.linalg.norm(gx)

        f_history.append(fx)
        grad_norm_history.append(grad_norm)

        # Check gradient convergence
        if grad_norm < tol_grad:
            break

        # Try Newton step
        try:
            Hx = hess(x)
            p = -np.linalg.solve(Hx + trust, gx)
            trust /= 2
            # ensure descent direction
            # if np.dot(p, gx) >= 0:
            #     raise np.linalg.LinAlgError
        except np.linalg.LinAlgError:
            # fallback to steepest descent
            print("Hessian not positive definite, using gradient descent")
            p = -gx
            trust *= 2
        # Backtracking line search
        alpha, fx = golden_section_search(
            lambda alpha: f(x + alpha * p),
            a=ls_bounds[0], b=ls_bounds[1], tol=1e-6, max_iter=100
        )

        # Update
        x_new = x + alpha * p
        step_norm = np.linalg.norm(x_new - x)
        x = x_new

        # Stall criterion
        if step_norm < tol_step:
            stall_count += 1
            if stall_count >= stall_iter:
                break
        else:
            stall_count = 0

        # print debug info
        if verbose:
            print(f"Iter {k}: f={fx:.6e}, grad_norm={grad_norm:.6e}, step_norm={step_norm:.6e}, alpha={alpha:.6f}")

    return x, grad_norm, f_history, grad_norm_history

In [ ]:
import numpy as np

from scipy.optimize import nnls


def optimal_a(d: np.ndarray,
              w: np.ndarray,
              target: np.ndarray,
              b: np.ndarray) -> np.ndarray:
    """
    Solve for a >= 0 minimizing
        sum_j w[j] * (sum_k a[k] * exp(-b[k] * d[j]**2) - target[j])**2

    Parameters
    ----------
    d : (J,) array
        Sample points x_j.
    w : (J,) array
        Quadrature weights.
    target : (J,) array
        Target values at each d[j].
    b : (K,) array
        Exponents b_k.

    Returns
    -------
    a : (K,) array
        Non-negative coefficient vector minimizing the weighted LS error.
    """
    # Build design matrix Φ[j,k] = exp(-b[k] * d[j]**2)
    # Note: np.outer(d**2, b) yields shape (J, K)
    Phi = np.exp(-np.outer(d**2, b))     # shape (J, K)

    # Incorporate weights by scaling rows
    W_sqrt = np.sqrt(w)                  # shape (J,)
    Phi_w = Phi * W_sqrt[:, None]        # shape (J, K)
    y_w = target * W_sqrt              # shape (J,)

    a, _ = nnls(Phi_w, y_w)

    return a

In [ ]:
# --- synthetic data ---------------------------------------------------
n_terms = 8               # number of (a_i, b_i) pairs
N       = 500               # data points

d, w = rectangle_rule(0.0, 2.0, N)
target = np.exp(-d)                  # any reference curve you like

b = np.linspace(-2, 14.5, n_terms)
a = optimal_a(d, w, target, np.exp(b))
print("Optimal a:", a)

means = np.concatenate((a, np.exp(b)))  # initial guess in log-space

# --- optimiser call ---------------------------------------------------
x, grad_norm, f_history, grad_norm_history = newton(
    f=obj,
    grad=grad,
    hess=hess,
    x0=means,
    tol_grad=1e-5,
    tol_step=1e-8,
    stall_iter=50,
    max_iter=3000,
    ls_bounds=(-2.0, 2.0)
)

print("Optimised parameters:", x)
print("Final gradient norm:", grad_norm)
print("Final objective value:", f_history[-1])

In [ ]:
import matplotlib.pyplot as plt
plt.plot(np.log(x[:n_terms]), label='a_i')
plt.plot(np.log(x[n_terms:]), label='b_i')
plt.xlabel('Parameter index')
plt.ylabel('Parameter value (log scale for a_i)')
plt.title('Optimised Parameters')
plt.legend()
plt.grid()
plt.show()
# Plot the convergence of the objective function

In [ ]:
g = grad(x)
H = hess(x)
d = np.linalg.solve(H, -g)  # Newton step
dir_obj = lambda alpha: obj(x + alpha * d)
alphas = np.linspace(-1.0, 1.0, 1000)

import matplotlib.pyplot as plt

plt.plot(alphas, [dir_obj(alpha) for alpha in alphas])
plt.xlabel("alpha")
plt.ylabel("Objective along Newton direction")
plt.title("Line search objective along Newton step")
plt.grid()
plt.show()

In [ ]:
N = 500               # data points

d, w = rectangle_rule(0.0, 5.0, N)
target = np.exp(-d)                  # any reference curve you like
f_min_all = []
min_b = -1.0
max_b = 1.5

# --- synthetic data ---------------------------------------------------
for n_terms in range(2, 16):               # number of (a_i, b_i) pairs

    b = np.linspace(min_b * 1.05, max_b * 1.1, n_terms)
    a = optimal_a(d, w, target, np.exp(b))
    # print("Optimal a:", a)

    means = np.concatenate((a, np.exp(b)))  # initial guess in log-space

    # --- optimiser call ---------------------------------------------------
    x, grad_norm, f_history, grad_norm_history = newton(
        f=obj,
        grad=grad,
        hess=hess,
        x0=means,
        tol_grad=1e-5,
        tol_step=1e-8,
        stall_iter=50,
        max_iter=1000,
        ls_bounds=(-2.0, 2.0)
    )
    min_b = min(np.log(x[n_terms:]))
    max_b = max(np.log(x[n_terms:]))
    print(f"Min a log: {min_b}, Max b log: {max_b}")
    print("Final objective value:", f_history[-1])
    f_min_all.append(f_history[-1])

In [ ]:
plt.plot(f_min_all)
plt.yscale('log')

In [ ]:
f_min_all

In [ ]:

# --- synthetic data ---------------------------------------------------

N = 500               # data points

d, w = rectangle_rule(0.0, 2.0, N)
target = np.exp(-d)                  # any reference curve you like
best_init = np.array([0,0])

for n_terms in range(1,10):                 # number of (a_i, b_i) pairs
    means = best_init  # initial mean for (a_i, b_i)
    cov = np.eye(2 * n_terms) * 4  # initial covariance matrix
    # --- optimiser call ---------------------------------------------------
    best, stats = cross_entropy_numba(
        d=d, target=target, w=w,
        n_terms=n_terms,
        max_iter=10000,
        pop_size=10000,
        elite_frac=0.1,
        mean=means,
        cov=cov
    )

    print("Best score :", stats["best_score"])
    print("Best params:", best)
    print("Iterations :", stats["iterations"])
    print("Runtime    :", stats["runtime"], "s")

    first_half = best[:n_terms]
    second_half = best[n_terms:]

    # Original equidistant indices (0 to n_terms-1), new indices from 0 to n_terms
    x_old = np.linspace(0, 1, n_terms)
    x_new = np.linspace(0, 1, n_terms + 1)

    interp_first = np.interp(x_new, x_old, first_half)
    interp_second = np.interp(x_new, x_old, second_half)

    best_init = np.hstack([interp_first, interp_second])

In [ ]:
best_init

In [ ]:
import matplotlib.pyplot as plt
plt.plot(stats["history"])
plt.xlabel("Iteration")
plt.ylabel("Best score")
plt.title("Convergence of the Cross-Entropy Method")
plt.grid()
plt.yscale('log')
plt.show()

In [ ]:
plt.plot(best[n_terms:], 'o-')
plt.xlabel("Index")
plt.ylabel("b_i (log scale)")
plt.title("Fitted b_i parameters")
plt.grid()
plt.show()

In [ ]:
from scipy.optimize import lsq_linear

# compute using least squares the coefficients for approximation with squared exponential kernels
# Construct the design matrix for squared exponential kernels
A = np.zeros((len(d), n_terms))
b_params = np.exp(best[n_terms:])

for i in range(n_terms):
    A[:, i] = np.exp(-b_params[i] * d**2)

# Solve the weighted least squares problem: minimize ||W(Aa - target)||^2
W = np.sqrt(w)
Aw = A * W[:, None]
tw = target * W

res = lsq_linear(Aw, tw, bounds=(0, np.inf))
squared_exponential_coeffs = res.x
print("Squared exponential coefficients:", np.log(squared_exponential_coeffs))

# compute fit with this coefficients
params = np.hstack([np.log(squared_exponential_coeffs), np.log(b_params)])

res = objective_numba(np.array([params]), d, target, w)
print("Objective value with least squares coefficients:", res[0])



In [ ]:
x, w = rectangle_rule(0.0, 2.0, 500)
f = lambda t: np.exp(-t)
a, b, info = fit_exp_sum_ce(5, x, w, f, iterations=2000, pop_size=1000)
print('a:', a)
print('b:', b)
print('best_score:', info.best_score)